# A Faster Horse

Just a quick script to check the stats of all my equines.

In [1]:
import json
from concurrent.futures import ProcessPoolExecutor, as_completed
from os import environ
from pathlib import Path
from typing import Any, Collection, Dict

import mutf8
import pandas as pd
from IPython.display import Markdown, display
from nbt import nbt, region

In [2]:
save_folder = Path(environ["SAVE_PATH"])

In [3]:
def get_name(entity: Dict[str, Any]) -> str:
    """Return the name (or identifier) of an entity

    Parameters
    ----------
    entity: dict
        The entity of interest

    Returns
    -------
    str
        The name and ID (or just the ID) of the entity
    """
    identifier = entity["id"]

    if "CustomName" not in entity.keys():
        return str(identifier)
    custom_name = json.loads(entity["CustomName"].value)
    try:
        name = custom_name["text"]  # 1.20.1-
    except TypeError:
        name = custom_name  # 1.20.3+
    return f"{name} ({identifier})"

In [4]:
def find_horseys(path: Path) -> list[Dict[str, Any]]:
    """Find and return all hawses
    in a given entities file

    Parameters
    ----------
    path : Path
        The path to the entities file

    Returns
    -------
    list of dict
        The attributes of all found horse
    """
    horses = []
    for chunk in region.RegionFile(path).iter_chunks():
        for entity in chunk["Entities"]:
            if entity["id"].value not in (
                "minecraft:horse",
                "minecraft:mule",
                "minecraft:donkey",
            ):
                continue
            attributes: Dict[str, Any] = {"name": get_name(entity)}
            attributes["wild"] = "Owner" not in entity
            attributes["location"] = tuple(
                int(v.value) for i, v in enumerate(entity["Pos"]) if i != 1
            )

            for attr in entity["Attributes"]:
                if any(
                    keyword in attr["Name"].value
                    for keyword in ("armor", "knockback", "follow_range")
                ):
                    continue
                attributes[attr["Name"].value.split(":")[-1].split(".")[-1]] = attr[
                    "Base"
                ].value

            horses.append(attributes)
    return horses

In [5]:
%%time
horse_data = []
futures = []
with ProcessPoolExecutor(max_workers=7) as executor:
    for path in sorted((save_folder / "entities").glob("*")):
        futures.append(executor.submit(find_horseys, path))

for result in as_completed(futures):
    horse_data.extend(result.result())

horse_dataframe = (
    pd.DataFrame(horse_data)
    .sort_values("movement_speed", ascending=False)
    .reset_index(drop=True)
)
horse_dataframe

CPU times: user 62.3 ms, sys: 52.7 ms, total: 115 ms
Wall time: 967 ms


,name,wild,location,max_health,movement_speed,jump_strength
0,Coltneighsegg (minecraft:horse),False,"(-1487, -3954)",24.000000,0.321808,0.863070
1,Juan Maneuel Fangiwhoa (minecraft:horse),False,"(-1519, -3957)",20.000000,0.316517,0.791625
2,Bugatti Neigh-ron (minecraft:horse),False,"(-1514, -3956)",19.000000,0.313177,0.881441
3,Paganeigh Hayra (minecraft:horse),False,"(-1523, -3973)",24.464658,0.311544,0.768651
4,Alfalfa Roam-mare-o (minecraft:horse),False,"(-1493, -3943)",19.000000,0.308744,0.839726
...,...,...,...,...,...,...
166,minecraft:donkey,True,"(-334, -1238)",22.000000,NaN,NaN
167,minecraft:donkey,True,"(-329, -1224)",24.000000,NaN,NaN
168,minecraft:donkey,True,"(-328, -1221)",19.000000,NaN,NaN
169,minecraft:donkey,True,"(-653, -4377)",23.000000,NaN,NaN


Since the last time I uploaded this script, I've done a bunch of husbandry,
so it's pretty gratifying to see that I've got **all five** of the five fastest horses. It's also cool to note that there's still room for improvement, as [max movement speed is 0.3375](https://minecraft.wiki/w/Horse#Movement_speed), and my fastest super~~car~~horse is only at 0.322.

Slicing out the wild ones, we have:

In [6]:
horse_dataframe.loc[~horse_dataframe.wild]

,name,wild,location,max_health,movement_speed,jump_strength
0,Coltneighsegg (minecraft:horse),False,"(-1487, -3954)",24.000000,0.321808,0.863070
1,Juan Maneuel Fangiwhoa (minecraft:horse),False,"(-1519, -3957)",20.000000,0.316517,0.791625
2,Bugatti Neigh-ron (minecraft:horse),False,"(-1514, -3956)",19.000000,0.313177,0.881441
3,Paganeigh Hayra (minecraft:horse),False,"(-1523, -3973)",24.464658,0.311544,0.768651
4,Alfalfa Roam-mare-o (minecraft:horse),False,"(-1493, -3943)",19.000000,0.308744,0.839726
7,Mareclaren 570LT (minecraft:horse),False,"(-1509, -3953)",23.000000,0.299387,0.756016
10,Ponticlack Firebird (minecraft:horse),False,"(-1502, -3946)",20.000000,0.287003,0.494529
12,Lacia Foalvia (minecraft:horse),False,"(-1524, -3945)",22.333333,0.276493,0.808521
14,Marecedes SLS (minecraft:horse),False,"(-1493, -3952)",23.000000,0.273471,0.825461
16,Lamborghi-neigh (minecraft:horse),False,"(-1487, -3970)",27.000000,0.271633,0.566055


It also might be worth trying my hand at mule-breeding again, since Samuel Camino is slower than the midpoint of Coltneighness crossed with a base donkey.

Moving onto jump strength:

In [7]:
horse_dataframe.sort_values("jump_strength", ascending=False).reset_index(drop=True)

,name,wild,location,max_health,movement_speed,jump_strength
0,minecraft:horse,True,"(-1229, -5374)",29.0,0.236095,0.924284
1,minecraft:horse,False,"(-983, -4277)",21.0,0.232124,0.910533
2,minecraft:horse,True,"(-462, -5377)",18.0,0.214107,0.909173
3,minecraft:horse,True,"(-7022, -1925)",24.0,0.250328,0.891672
4,minecraft:horse,True,"(-788, -5326)",24.0,0.225324,0.888814
...,...,...,...,...,...,...
166,minecraft:donkey,True,"(-334, -1238)",22.0,NaN,NaN
167,minecraft:donkey,True,"(-329, -1224)",24.0,NaN,NaN
168,minecraft:donkey,True,"(-328, -1221)",19.0,NaN,NaN
169,minecraft:donkey,True,"(-653, -4377)",23.0,NaN,NaN


In [8]:
_.loc[~_.wild]  # type: ignore

,name,wild,location,max_health,movement_speed,jump_strength
1,minecraft:horse,False,"(-983, -4277)",21.000000,0.232124,0.910533
5,Bugatti Neigh-ron (minecraft:horse),False,"(-1514, -3956)",19.000000,0.313177,0.881441
9,Mareclaren P1 (minecraft:horse),False,"(-1493, -3946)",23.000000,0.270267,0.864608
10,Coltneighsegg (minecraft:horse),False,"(-1487, -3954)",24.000000,0.321808,0.863070
13,minecraft:horse,False,"(-1010, -5143)",25.000000,0.212723,0.855987
14,Ca-Mare-o (minecraft:horse),False,"(-1500, -3958)",21.000000,0.232040,0.852316
19,Horsche 924 (minecraft:horse),False,"(-1495, -3952)",19.000000,0.238490,0.840111
20,Alfalfa Roam-mare-o (minecraft:horse),False,"(-1493, -3943)",19.000000,0.308744,0.839726
23,Marecedes SLS (minecraft:horse),False,"(-1493, -3952)",23.000000,0.273471,0.825461
27,Lacia Foalvia (minecraft:horse),False,"(-1524, -3945)",22.333333,0.276493,0.808521


This isn't really something I'm looking at investing in right now, but it's cool to see that two of my fastest horses are also among my best jumpers.